In [73]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Set your Kaggle API credentials (make sure to upload your kaggle.json file to your notebook)
# os.environ["KAGGLE_USERNAME"] = "ajfunari"
# os.environ["KAGGLE_KEY"] = "cb66eed558fe1d91db565e0d279b2fdb"

# # Create a Kaggle API object
api = KaggleApi()
api.authenticate()

# # Specify the dataset you want to download
dataset_name = "tawsifurrahman/covid19-radiography-database"

# # Set the destination folder for the downloaded dataset
current_directory = os.getcwd()
download_path = "/input"
path = current_directory + download_path
print(path)

# # Download the dataset
api.dataset_download_files(dataset_name, path=path, unzip=True)  # 800 MB

c:\Users\flyin\OneDrive\Desktop\Github\Xray_Classification/input


In [97]:
from torchvision.io import read_image
from torch.utils.data import Dataset
import pandas as pd

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_excel(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [98]:
import cv2
import matplotlib.pyplot as plt

# current working directory
current_directory = os.getcwd()

# image directory
img_folder = '/input/COVID-19_Radiography_Dataset/COVID/images'
folder_path = current_directory + img_folder
image_files = os.listdir(folder_path)

# visualize one image
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)
    # plt.imshow(image)
    break

# metadata directory
annotated_file = '/input/COVID-19_Radiography_Dataset/COVID.metadata.xlsx'
annotated_file_path = current_directory + annotated_file
df = pd.read_excel(annotated_file_path)

In [107]:
data = CustomImageDataset(annotated_file_path, folder_path)
# print(data.__getitem__(1))